In [86]:
from dotenv import load_dotenv
import os

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [87]:
llm_config = {
    "model": "gpt-3.5-turbo",
    "api_key": OPENAI_API_KEY,
}

In [88]:
import chess
import chess.svg

In [89]:
board = chess.Board()
made_move = False

In [90]:
from typing import Annotated


def get_legal_moves() -> Annotated[str, "A list of legal moves in UCI format"]:
    return "Possible moves: " + ",".join(str(move) for move in board.legal_moves)

In [91]:
def make_move(
    m: Annotated[str, "A move in UCI format."]
) -> Annotated[str, "Result of the move."]:
    move = chess.Move.from_uci(m)
    board.push_uci(str(move))
    global made_move
    made_move = True

    # Display the board.
    display(
        chess.svg.board(
            board,
            arrows=[(move.from_square, move.to_square)],
            fill={move.from_square: "gray"},
            size=200,
        )
    )

    # Get the piece name.
    piece = board.piece_at(move.to_square)
    if piece is None:
        return "Invalid move."
    piece_symbol = piece.unicode_symbol()
    piece_name = (
        chess.piece_name(piece.piece_type).capitalize()
        if piece_symbol.isupper()
        else chess.piece_name(piece.piece_type)
    )
    return (
        f"Moved {piece_name} ({piece_symbol}) from "
        f"{chess.SQUARE_NAMES[move.from_square]} to "
        f"{chess.SQUARE_NAMES[move.to_square]}."
    )

In [92]:
from autogen import ConversableAgent

In [93]:
# Player white agent
player_white = ConversableAgent(
    name="player_white",
    system_message="You are a chess player and you play as white. "
    "First call get_legal_moves(), to get a list of legal moves. "
    "Then call make_move(move) to make a move.",
    llm_config=llm_config,
)

# Player black agent
player_black = ConversableAgent(
    name="player_black",
    system_message="You are a chess player and you play as black. "
    "First call get_legal_moves(), to get a list of legal moves. "
    "Then call make_move(move) to make a move.",
    llm_config=llm_config,
)

In [94]:
def check_made_move(msg):
    global made_move
    if made_move:
        made_move = False
        return True
    return False

In [95]:
board_proxy = ConversableAgent(
    name="board_proxy",
    llm_config=False,
    is_termination_msg=check_made_move,
    default_auto_reply="Please make a move.",
    human_input_mode="NEVER",
)

In [ ]:
from autogen import register_function

for caller in [player_white, player_black]:
    register_function(
        get_legal_moves,
        caller=caller,
        executor=board_proxy,
        description="Get a list of legal moves for the current board state.",
    )
    register_function(
        make_move,
        caller=caller,
        executor=board_proxy,
        description="Make a move on the board.",
    )

In [ ]:
player_black.llm_config["tools"]

In [98]:
player_white.register_nested_chats(
    trigger=player_black,
    chat_queue=[
        {
            "sender": board_proxy,
            "recipient": player_white,
            "summary_method": "last_msg",
        }
    ],
)

player_black.register_nested_chats(
    trigger=player_white,
    chat_queue=[
        {
            "sender": board_proxy,
            "recipient": player_black,
            "summary_method": "last_msg",
        }
    ],
)

In [ ]:
chat_result = player_black.initiate_chat(
    player_white,
    message="Let's play chess! Your move.",
    max_turns=20,
)